# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
import nltk
nltk.download(['punkt', 'wordnet', 'omw-1.4'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maxsokolov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maxsokolov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/maxsokolov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table('DisasterMeassages', con=engine)

In [15]:
df = df[df.related != 2]

In [24]:
X = df.message.values
Y = df.iloc[:,3:]

### 2. Write a tokenization function to process your text data

In [25]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff1e861ef70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
y_pred = pipeline.predict(X_test)

In [29]:
y_pred[:,0]

array([1, 0, 1, ..., 1, 1, 1])

In [30]:
y_test['related'].value_counts()

1    4989
0    1518
Name: related, dtype: int64

In [31]:
y_cols = list(df.columns)[3:]

In [32]:
y_cols

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [33]:
i = 0
for col in y_cols:
    print(col)
    print(i)
    classification_report(y_true=y_test[col], y_pred=y_pred[:,i], output_dict=True)
    i += 1

related
0
request
1
offer
2
aid_related
3
medical_help
4
medical_products
5
search_and_rescue
6
security
7
military
8
child_alone
9
water
10


/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Use

food
11
shelter
12
clothing
13
money
14
missing_people
15
refugees
16
death
17
other_aid
18
infrastructure_related
19
transport
20
buildings
21
electricity
22
tools
23
hospitals
24
shops
25


/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Use

aid_centers
26
other_infrastructure
27
weather_related
28
floods
29
storm
30
fire
31
earthquake
32
cold
33
other_weather
34
direct_report
35


/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
classification_report(y_true=y_test['related'], y_pred=y_pred[:,0], output_dict=True)

{'0': {'precision': 0.76,
  'recall': 0.2628458498023715,
  'f1-score': 0.39060205580029367,
  'support': 1518},
 '1': {'precision': 0.8129388164493481,
  'recall': 0.9747444377630787,
  'f1-score': 0.8865190046486191,
  'support': 4989},
 'accuracy': 0.8086675887505763,
 'macro avg': {'precision': 0.786469408224674,
  'recall': 0.6187951437827252,
  'f1-score': 0.6385605302244564,
  'support': 6507},
 'weighted avg': {'precision': 0.8005888666460423,
  'recall': 0.8086675887505763,
  'f1-score': 0.7708279137692956,
  'support': 6507}}

### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
parameters = {
        'clf__estimator__n_estimators': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
#,'clf__ccp_alpha': [0.0, 0.1, 0.3]
#'clf__estimator__criterion': ['gini', 'entropy'],
#,'clf__estimator__min_samples_split': [2, 3, 4]

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
cv.fit(X_train, y_train)

/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 't

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ff1e861ef70>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]})

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [38]:
cv.cv_results_

{'mean_fit_time': array([170.33288474, 395.48537555]),
 'std_fit_time': array([31.55826148, 45.797834  ]),
 'mean_score_time': array([ 5.76526484, 12.18146582]),
 'std_score_time': array([1.40410826, 2.07825064]),
 'param_clf__estimator__n_estimators': masked_array(data=[50, 100],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__n_estimators': 50},
  {'clf__estimator__n_estimators': 100}],
 'split0_test_score': array([0.24020487, 0.23457106]),
 'split1_test_score': array([0.23232582, 0.23591189]),
 'split2_test_score': array([0.22848361, 0.23181352]),
 'split3_test_score': array([0.2505123 , 0.24743852]),
 'split4_test_score': array([0.25512295, 0.24718238]),
 'mean_test_score': array([0.24132991, 0.23938347]),
 'std_test_score': array([0.01021696, 0.00660639]),
 'rank_test_score': array([1, 2], dtype=int32)}

In [39]:
pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50)))
    ])

In [41]:
pipeline1.fit(X_train, y_train)

/Users/maxsokolov/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff1e861ef70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=50)))])

In [42]:
y_pred = pipeline1.predict(X_test)

In [43]:
classification_report(y_true=y_test['related'], y_pred=y_pred[:,0], output_dict=True)

{'0': {'precision': 0.7588028169014085,
  'recall': 0.2839262187088274,
  'f1-score': 0.41323106423777567,
  'support': 1518},
 '1': {'precision': 0.816972554302071,
  'recall': 0.9725395870916015,
  'f1-score': 0.8879941434846266,
  'support': 4989},
 'accuracy': 0.8118948824343015,
 'macro avg': {'precision': 0.7878876856017398,
  'recall': 0.6282329029002145,
  'f1-score': 0.6506126038612011,
  'support': 6507},
 'weighted avg': {'precision': 0.803402297444194,
  'recall': 0.8118948824343015,
  'f1-score': 0.7772379802301745,
  'support': 6507}}

### 9. Export your model as a pickle file

In [44]:
import pickle

pickle.dump(pipeline1, open('pipeline1.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.